## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup

import pandas as pd
import requests
import gmaps


# Import API key

from config import g_key


# Configure gmaps API key

gmaps.configure(api_key=g_key)

In [3]:
# Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,33.46,75,75,13.80,broken clouds
1,1,Balaghat,IN,21.8000,80.1833,83.03,79,70,3.40,broken clouds
2,2,Saint Anthony,US,45.0205,-93.2180,73.94,74,75,11.50,broken clouds
3,3,Lagoa,PT,39.0500,-27.9833,68.47,68,100,5.26,overcast clouds
4,4,Charters Towers,AU,-20.1000,146.2667,51.93,56,0,9.10,clear sky


In [4]:
# Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [5]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Balaghat,IN,21.8000,80.1833,83.03,79,70,3.40,broken clouds
2,2,Saint Anthony,US,45.0205,-93.2180,73.94,74,75,11.50,broken clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,71.19,71,24,15.30,few clouds
6,6,Cravinhos,BR,-21.3403,-47.7294,72.32,40,18,8.84,few clouds
11,11,Atambua,ID,-9.1061,124.8925,71.64,80,65,5.61,broken clouds
13,13,Butaritari,KI,3.0707,172.7902,82.08,78,13,17.45,light rain
15,15,Rikitea,PF,-23.1203,-134.9692,72.30,79,24,4.63,light rain
16,16,Zhangye,CN,38.9342,100.4517,78.64,45,100,8.75,overcast clouds
18,18,Sinnamary,GF,5.3833,-52.9500,78.60,86,90,3.13,overcast clouds
20,20,Hasaki,JP,35.7333,140.8333,75.60,93,0,7.92,clear sky


In [15]:
# Determine if there are any empty rows.

preferred_cities_df.notnull().count()

City_ID                364
City                   364
Country                364
Lat                    364
Lng                    364
Max Temp               364
Humidity               364
Cloudiness             364
Wind Speed             364
Current Description    364
dtype: int64

In [16]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Balaghat,IN,21.8000,80.1833,83.03,79,70,3.40,broken clouds
2,2,Saint Anthony,US,45.0205,-93.2180,73.94,74,75,11.50,broken clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,71.19,71,24,15.30,few clouds
6,6,Cravinhos,BR,-21.3403,-47.7294,72.32,40,18,8.84,few clouds
11,11,Atambua,ID,-9.1061,124.8925,71.64,80,65,5.61,broken clouds


In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


# Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Balaghat,IN,83.03,broken clouds,21.8000,80.1833,
2,Saint Anthony,US,73.94,broken clouds,45.0205,-93.2180,
5,Saint-Philippe,RE,71.19,few clouds,-21.3585,55.7679,
6,Cravinhos,BR,72.32,few clouds,-21.3403,-47.7294,
11,Atambua,ID,71.64,broken clouds,-9.1061,124.8925,
13,Butaritari,KI,82.08,light rain,3.0707,172.7902,
15,Rikitea,PF,72.30,light rain,-23.1203,-134.9692,
16,Zhangye,CN,78.64,overcast clouds,38.9342,100.4517,
18,Sinnamary,GF,78.60,overcast clouds,5.3833,-52.9500,
20,Hasaki,JP,75.60,clear sky,35.7333,140.8333,


In [18]:
# Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    
    # Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
    # Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# Drop the rows where there is no Hotel Name.

import numpy as np

hotel_df["Hotel Name"].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Balaghat,IN,83.03,broken clouds,21.8000,80.1833,Hotel celebration
2,Saint Anthony,US,73.94,broken clouds,45.0205,-93.2180,"Holiday Inn Express Roseville-St. Paul, an IHG..."
5,Saint-Philippe,RE,71.19,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Cravinhos,BR,72.32,few clouds,-21.3403,-47.7294,Ônix Inn Hotel Cravinhos
11,Atambua,ID,71.64,broken clouds,-9.1061,124.8925,Kost Alfa Flat


In [26]:
# Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""


# Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))